In [1]:
# Imports, nothing to see here
import random
import sys,os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sys.path.insert(0, os.path.abspath('..'))
from spreadsheet.cell import Cell
from spreadsheet.arraySpreadsheet import ArraySpreadsheet
from spreadsheet.linkedlistSpreadsheet import LinkedListSpreadsheet
from spreadsheet.csrSpreadsheet import CSRSpreadsheet
from spreadsheet.baseSpreadsheet import BaseSpreadsheet
from time import perf_counter

In [2]:
def create_random_cells(num_cells: int, max_col:int, max_row:int) -> list[Cell]:
    ret = ""
    ret += str(max_row-1)+" "+ str(max_col-1)+ " "+str(round(random.uniform(0.1,9.9), ndigits=2)) +"\n"
    for _ in range(num_cells):
        row = random.randint(0, max_col-1)
        col = random.randint(0, max_row-1)
        val = round(random.uniform(0.1,9.9), ndigits=2)
        ret += str(row) +" "+  str(col) + " "+str(val)+"\n"
    return ret

def write_to_file(cells_to_generate:int, max_col:int, max_row:int):
    file = open("sample_data.txt","w")
    file.write(create_random_cells(cells_to_generate, max_col, max_row))

def display_array(array: ArraySpreadsheet):
    arr =array.array 
    for x in arr:
        for y in x:
            if(y.val == None):
                print("("+str(y.row),str(y.col)+" N)", end = " ")
            else:
                print(y, end = " ")
        print() #newline
    print("finished")

def get_lCells()->list[Cell]:
    """
    basically read from the file and create a list of cells from the data from the file
    """
    try:
        lcells = []
        data_time = perf_counter()
        dataFile = open("sample_data.txt", 'r')
        for line in dataFile:
            values = line.split()
            currRow = int(values[0])
            currCol = int(values[1])
            currVal = float(values[2])
            currCell = Cell(currRow, currCol, currVal)
            # each line contains a cell
            lcells.append(currCell)
        dataFile.close()
        data_time_close = perf_counter()
        return lcells
    except FileNotFoundError as e:
        print("Data file doesn't exist.")
    pass

def set_up_cells(cells_to_generate, max_col, max_row):
    write_to_file(cells_to_generate, max_col, max_row)
    return get_lCells()

def test_buildSpreadSheet(spreadSheet:BaseSpreadsheet,lCells:list[Cell]):
    start_time = perf_counter()
    spreadSheet.buildSpreadsheet(lCells)
    end_time = perf_counter()
    return end_time-start_time

def test_entries(spreadSheet:BaseSpreadsheet):
    start_time = perf_counter()
    spreadSheet.entries()
    end_time = perf_counter()
    return end_time-start_time

def test_update(spreadSheet:BaseSpreadsheet, rowIndex: int, colIndex:int, val:float):
    start_time = perf_counter()
    spreadSheet.update(rowIndex, colIndex,val)
    end_time = perf_counter()
    return end_time-start_time

def test_append_row(spreadSheet:BaseSpreadsheet):
    start_time = perf_counter()
    spreadSheet.appendRow()
    end_time = perf_counter()
    return end_time-start_time

def test_append_col(spreadSheet:BaseSpreadsheet):
    start_time = perf_counter()
    spreadSheet.appendCol()
    end_time = perf_counter()
    return end_time-start_time

def test_insert_row(spreadSheet:BaseSpreadsheet, rowIndex:int):
    start_time = perf_counter()
    spreadSheet.insertRow(rowIndex)
    end_time = perf_counter()
    return end_time-start_time

def test_insert_col(spreadsheet:BaseSpreadsheet, colIndex:int):
    start_time = perf_counter()
    spreadsheet.insertCol(colIndex)
    end_time = perf_counter()
    return end_time-start_time

def find(spreadSheet:BaseSpreadsheet, value:float):
    start_time = perf_counter()
    spreadSheet.find(value)
    end_time = perf_counter()
    return end_time-start_time


In [3]:
small_spreadsheets= []
for _ in range(10):
    small_spreadsheets.append(ArraySpreadsheet())
small_lCells = []
for _ in range(10):
    small_lCells.append(set_up_cells(_*5000, 25, 25))

In [26]:
small_spreadsheet_tests = []
small_spreadsheet_tests.append([test_buildSpreadSheet(small_spreadsheets[x], small_lCells[x])*1000 for x in range(10)])
small_spreadsheet_tests.append([test_append_col(small_spreadsheets[x])*1000 for x in range(10)])
small_spreadsheet_tests.append([test_append_row(small_spreadsheets[x])*1000 for x in range(10)])
small_spreadsheet_tests.append([test_insert_col(small_spreadsheets[x], x)*1000 for x in range(10)])
small_spreadsheet_tests.append([test_insert_row(small_spreadsheets[x], x)*1000 for x in range(10)])
small_spreadsheet_tests.append([test_entries(small_spreadsheets[x])*1000 for x in range(10)])



In [27]:
df = pd.DataFrame(small_spreadsheet_tests)
df.columns = [500*x for x in range(1,11)]
df.index = ["build_spreadsheet","append_col", "append_row", "insert_col", "insert_row","entries"]  
df

,500,1000,1500,2000,2500,3000,3500,4000,4500,5000
build_spreadsheet,0.007083,5.481500,8.888958,11.973916,13.268458,15.555000,16.429709,19.446625,20.594042,23.989667
append_col,0.013917,0.019667,0.010250,0.009958,0.009333,0.473667,0.008833,0.009833,0.009792,0.008708
append_row,0.012000,0.011333,0.011208,0.011000,0.011083,0.021041,0.011000,0.011791,0.010542,0.010750
insert_col,0.128083,0.127791,0.145167,0.123917,0.154625,0.124917,0.115084,0.116333,0.111042,0.250334
insert_row,0.104875,0.100542,0.101416,0.096292,0.091875,0.103792,0.089708,0.088833,0.088625,0.085083
entries,0.110667,1.901084,2.059291,2.033250,2.035000,1.772209,1.635209,1.680792,1.972750,1.943250
